In [1]:
import json
from pathlib import Path
import time
import datetime
import numpy as np
import pandas as pd
from loguru import logger
import pygmo as pg
from IPython.display import clear_output

from solarmed_optimization import (EnvironmentVariables,
                                   ProblemParameters,
                                   ProblemSamples,
                                   AlgorithmParameters,
                                   PopulationResults)

from solarmed_optimization.utils.initialization import problem_initialization, InitialStates
from solarmed_optimization.utils.evaluation import evaluate_optimization
from solarmed_optimization.utils.serialization import OptimizationResults
from solarmed_optimization.utils.visualization import generate_visualizations
from solarmed_optimization.problems.pygmo import MinlpProblem

# auto reload modules
%load_ext autoreload
%autoreload 2
logger.disable("phd_visualizations")

#%% Constants
# Paths definition
base_output_path: Path = Path("../results")
data_path: Path = Path("../data")
fsm_data_path: Path = Path("../results/fsm_data")
date_str: str = "20180921_20180928" # "20230707_20230710" # '20230630' '20230703'
n_islands: int = 10

if not base_output_path.exists():
    base_output_path.mkdir()

# Either load parameters from json or create a new instance
# with open(output_path / "20230703_eval_at_20250105/gaco/problem_params.json") as f:
#     problem_params = ProblemParameters(**json.load(f))
problem_params: ProblemParameters = ProblemParameters(
    optim_window_time=8*3600, # 8 hours
)
optim_params: AlgorithmParameters = AlgorithmParameters(
    pop_size=16, # 32
    n_gen=3,
    seed_num=32
)
initial_states: InitialStates = InitialStates(Tts_h=[90, 80, 70], 
                                              Tts_c=[70, 60, 50]) 

problem_data = problem_initialization(problem_params=problem_params,
                                      date_str=date_str,
                                      data_path=data_path,
                                      initial_states=initial_states)

ps: ProblemSamples = problem_data.problem_samples
pp: ProblemParameters = problem_data.problem_params
df: pd.DataFrame = problem_data.df
model = problem_data.model

# df_mods: list[pd.DataFrame] = []
df_hors: list[pd.DataFrame] = []
df_sim: pd.DataFrame = None

# Setup optimization algorithm / computation strategy
algorithm = pg.gaco(gen=optim_params.n_gen, ker=optim_params.pop_size, seed=optim_params.seed_num)
algo = pg.algorithm(algorithm)
algo.set_verbosity(1) # regulates both screen and log verbosity

# island = pg.ipyparallel_island()
algo_params =  {
    "gen": optim_params.n_gen, 
    "ker": optim_params.pop_size, 
    "seed": optim_params.seed_num
}
algo_id = "gaco"
metadata: dict[str, str] = {"date_str": date_str, "algo_id": algo_id}
output_path = base_output_path / f"{date_str}_eval_at_{datetime.datetime.now(tz=datetime.timezone.utc):%Y%m%d}" / algo_id
output_path.mkdir(parents=True, exist_ok=True)


2025-01-17 13:38:21.815 | WARNING  | solarmed_optimization.utils.initialization:problem_initialization:103 - External environment data (20220524 - 20240524) (seawater temperature and salinity) is not available for the data period (20180921 - 20180928). Using data for same month and days but from closest year 2022.


In [ ]:
output_path.mkdir(parents=True, exist_ok=True)
opt_step_idx: int = 0
max_opt_steps: int = (len(df)-pp.idx_start-ps.optim_window_samples) // ps.n_evals_mod_in_opt_step - 1
idx_mod = pp.idx_start
initial_time = time.time()
opt_step_idx = 0
# for opt_step_idx in range(0, max_opt_steps):
hor_span = (idx_mod+1, idx_mod+1+ps.n_evals_mod_in_hor_window)

# Optimization step `opt_step_idx`
print("")
print(f"Optimization step {opt_step_idx+1}/{max_opt_steps}")

# 1. Initialize the problem instance
## Environment variables predictions
ds = df.iloc[hor_span[0]:hor_span[1]]
env_vars: EnvironmentVariables = EnvironmentVariables(
    I=ds['I'].values,
    Tamb=ds['Tamb'].values,
    Tmed_c_in=ds['Tmed_c_in'].values,
    cost_w=np.ones((ps.n_evals_mod_in_hor_window, )) * pp.env_params.cost_w,
    cost_e=np.ones((ps.n_evals_mod_in_hor_window, )) * pp.env_params.cost_e,
)


# print(prob)

# Initialize population
# pop = pg.population(prob, size=pop_size, seed=seed_num)
# display(pop)

# Initialize optimization algorithm
# algo = pg.algorithm(pg.gaco(ker=pop_size, gen=100))
# print(f"Running {algo.get_name()}")
# algo.set_verbosity(1) # regulates both screen and log verbosity




Optimization step 1/198


In [5]:
## Initialize problem
problem = MinlpProblem(
    model=model, 
    sample_time_opt=pp.sample_time_opt,
    optim_window_time=pp.optim_window_time,
    env_vars=env_vars,
    dec_var_updates=pp.dec_var_updates,
    fsm_valid_sequences=pp.fsm_valid_sequences,
    fsm_data_path=fsm_data_path,
    use_inequality_contraints=False
)


In [4]:
from solarmed_optimization.utils.initialization import generate_integer_pop

paths_df = problem.fsm_med_data.paths_df

generate_integer_pop(
    model=model, pp=pp, pop_size=50,
    paths_from_state_df=paths_df[paths_df["0"] == 0]
)


AttributeError: 'DecisionVariablesUpdates' object has no attribute 'sf_active'

In [6]:
prob = pg.problem(problem)


In [27]:
from solarmed_modeling.fsms import MedState, SfTsState
from solarmed_optimization.utils.initialization import generate_population
from solarmed_optimization.utils import decision_variables_to_decision_vector
# Initial population for first optimization step
model.med_state = MedState.OFF
model.sf_ts_state = SfTsState.IDLE
paths_df = problem.fsm_med_data.paths_df

pop_dec_vars = generate_population(model=model, pp=pp, problem=problem,
                                   pop_size=optim_params.pop_size, 
                                   prob=prob,
                                   paths_from_state_df=paths_df[paths_df["0"] == model.med_state.value])

print("Initial population in first optimization step")
for x_idx, x in enumerate(pop_dec_vars):
    print(f"{x_idx:02d}: {x.med_mode}, {x.qts_src}")
    
# Imagine that after this step, we are in some other states
# Then, the new initial population should be generated based on this new state
# we use the previous best solution to initialize the real variables, which will
# necesarily match one of the generated integer profiles
best_x = decision_variables_to_decision_vector(pop_dec_vars[1])
model.med_state = MedState.ACTIVE
model.sf_ts_state = SfTsState.HEATING_UP_SF

pop_dec_vars = generate_population(model=model, pp=pp, problem=problem,
                                   pop_size=optim_params.pop_size,
                                   dec_vec=best_x,
                                   paths_from_state_df=paths_df[paths_df["0"] == model.med_state.value])

print("Initial population in first optimization step")
for x_idx, x in enumerate(pop_dec_vars):
    print(f"{x_idx:02d}: {x.med_mode}, {x.qts_src}")


Initial population in first optimization step
00: [1 1 1 1 0 0 0 0 0 0], [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
01: [1 1 1 1 0 1 1 1 1 1], [ 0.9  6.   7.9  1.7  2.3 13.5  7.4  5.3  6.5  0.9]
02: [1 1 1 1 1 0 0 0 0 0], [ 0.   6.   7.9  1.7  2.3 13.5  7.4  5.3  6.5  0.9]
03: [1 1 1 1 1 0 1 1 1 1], [ 0.   0.   7.9  1.7  2.3 13.5  7.4  5.3  6.5  0.9]
04: [1 1 1 1 1 1 0 0 0 0], [ 0.   0.   0.   1.7  2.3 13.5  7.4  5.3  6.5  0.9]
05: [1 1 1 1 1 1 0 1 1 1], [ 0.   0.   0.   0.   2.3 13.5  7.4  5.3  6.5  0.9]
06: [1 1 1 1 1 1 1 0 0 0], [ 0.   0.   0.   0.   0.  13.5  7.4  5.3  6.5  0.9]
07: [1 1 1 1 1 1 1 0 1 1], [0.  0.  0.  0.  0.  0.  7.4 5.3 6.5 0.9]
08: [1 1 1 1 1 1 1 1 0 0], [0.  0.  0.  0.  0.  0.  0.  5.3 6.5 0.9]
09: [1 1 1 1 1 1 1 1 1 1], [0.  0.  0.  0.  0.  0.  0.  0.  6.5 0.9]
10: [0 1 1 1 1 0 0 0 0 0], [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9]
11: [0 1 1 1 1 0 1 1 1 1], [ 0.9  6.   7.9  1.7  2.3 13.5  7.4  5.3  6.5  0.9]
12: [0 1 1 1 1 1 0 0 0 0], [ 0.   6.   7.9  1.7  2.3 13.5  7.4  5.

In [9]:
isl = pg.island(algo = algo,  prob = prob, size = optim_params.pop_size) # udi=island

isl.evolve()
print(isl)

start_time = time.time()
while isl.status == pg.evolve_status.busy:
    time.sleep(5)
    print(f"Elapsed time: {time.time() - start_time:.0f}")
    # print(f"Current evolution results | Best fitness: {pop_current.champion_f[0]}, \nbest decision vector: {pop_current.champion_x}")
optim_eval_elapsed_time = int(time.time() - start_time)
print(f"Completed evolution! Took {time.time() - start_time:.0f} seconds") #| Best fitness: {pop_current.champion_f[0]}, \nbest decision vector: {pop_current.champion_x}")

problem = isl.get_population().problem.extract(object)
# Evaluate optimization and move `model` to the next step
# Update, only evaluate the best in the population, otherwise it takes too much space 
df_hor, df_sim, model = evaluate_optimization(
    df_sim=df_sim, 
    pop=[ isl.get_population().get_x()[isl.get_population().best_idx()] ],
    best_idx=0,
    env_vars=env_vars, problem=problem,
    problem_data=problem_data, idx_mod=idx_mod
)
# if opt_step_idx > 0:
#     df_hors[-1] = df_hors[-1][step_results.best_idx_per_gen[-1]] # Only keep the dataframe from the best individual from past steps
df_hors.append(df_hor[0])

pop_results: PopulationResults = PopulationResults.initialize(
    problem=problem,
    pop_size=optim_params.pop_size,
    n_gen=optim_params.n_gen,
    elapsed_time=optim_eval_elapsed_time,
)

# Export results
OptimizationResults(
    metadata=metadata,
    problem_params=problem_params,
    algo_log=isl.get_algorithm().extract( getattr(pg, algo_id) ).get_log(),
    df_hor=df_hor[0],
    df_sim=df_sim,
    pop_results=pop_results,
    algo_params=algo_params,
    figs=generate_visualizations(problem=problem, df_hors=df_hors, df_sim=df_sim, 
                                 problem_data=problem_data, metadata=metadata, 
                                 pop_results=pop_results)
).dump(output_path=output_path, step_idx=opt_step_idx)

# Finally, increase counter
idx_mod += ps.n_evals_mod_in_opt_step

print(f"Elapsed time: {time.time() - initial_time:.0f}")
print("")

# pop = algo.evolve(pop) 
# print(f"After initiating evolution: \n{isl}")

# uda=algo.extract(pg.gaco)
# print(f"Completed evolution, best fitness: {pop.champion_f[0]}, \nbest decision vector: {pop.champion_x}")

# print(uda.get_log())
# for iter_log in uda.get_log():
#     print(iter_log)

# Archipielago variant
# archi = pg.archipelago(n=n_islands,algo=algo, prob=prob, pop_size=pop_size, seed=seed_num)
# archi.evolve() 
# print(archi)


Island name: Multiprocessing island
	C++ class name: pybind11::object

	Status: busy

Extra info:
	Using a process pool: yes
	Number of processes in the pool: 16

Algorithm: GACO: Ant Colony Optimization

Problem: SolarMED MINLP problem

Replacement policy: Fair replace

Selection policy: Select best

Population size: 16
	Champion decision vector: [0, 7.10908, 1.55678, 1.63947, 0.761935, ... ]
	Champion fitness: [4.7806]


   Gen:        Fevals:          Best:        Kernel:        Oracle:            dx:            dp:
      1              0         4.7806             16              0          732.2        0.62313
Elapsed time: 5
      2             16        4.55498             16              0        729.546       0.509918
Elapsed time: 10
Elapsed time: 15
      3             48        4.34941             16              0        468.295       0.356965
Elapsed time: 20
Completed evolution! Took 20 seconds


TypeError: evaluate_optimization() missing 1 required positional argument: 'model'

In [17]:
# Archipielago option
while archi.status == pg.evolve_status.busy:
    clear_output()
    print(archi)
    print(f"Current evolution results per island | Best fitness: {archi.get_champions_f()}, \nbest decision vector: {archi.get_champions_x()}")
    
    time.sleep(5)

print(f"Evolution completed | Best fitness: {archi.get_champions_f()}, \nbest decision vector: {archi.get_champions_x()}")


Evolution completed | Best fitness: [array([-2.3]), array([-2.2]), array([-2.3]), array([-2.2]), array([-2.3]), array([-2.3]), array([-2.2]), array([-2.2]), array([-2.2]), array([-2.2])], 
best decision vector: [array([ 0. ,  0. ,  2.8,  0.6,  8.6,  8.1,  8.5,  2.6,  4.5,  4.6,  7.8,
        8.4,  0. , 18.4, 16.8, 17.2, 12.9, 11.1,  0. ,  0. ,  0. , 16.8,
       41.3, 17.2,  0. ,  0. ,  0. ,  5.5,  5.9,  3.5,  0. ,  0. ,  0. ,
       52.1, 60.7, 61.5,  0. ,  0. ,  0. ,  8.9,  5.4,  2. ,  0. ,  1. ,
        1. ,  0. ,  1. ,  1. ,  0. ,  1. ,  1. ,  1. ,  1. ,  1. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  1. ,  0. ,  2. ,  2. ,  2. ]), array([ 0. ,  0. ,  1.6,  0.9,  2.3,  4.7,  1.8,  2.5,  5.4,  3.9,  4.1,
        7.7,  0. ,  3.6,  5.2, 18.7,  1.1, 18.7,  0. ,  0. ,  0. , 22.1,
       28. , 38.8,  0. ,  0. ,  0. ,  0.8,  6.5,  1.9,  0. ,  0. ,  0. ,
       12.8, 57.1,  0.7,  0. ,  0. ,  0. , 20.7, 22.1, 26.8,  0. ,  0. ,
        1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  1. ,  

In [4]:
# Initialize optimization algorithm
algo = pg.algorithm(pg.gaco(ker=pop_size, gen=10))
print(f"Running {algo.get_name()}")
algo.set_verbosity(1) # regulates both screen and log verbosity

pop = algo.evolve(pop) 

# Extract results of evolution
uda=algo.extract(pg.gaco)
print(f"Completed evolution, best fitness: {pop.champion_f[0]}, \nbest decision vector: {pop.champion_x}")

# print(uda.get_log())
for iter_log in uda.get_log():
    print(iter_log)


Running GACO: Ant Colony Optimization

   Gen:        Fevals:          Best:        Kernel:        Oracle:            dx:            dp:
      1              0      -0.533408              3              0        13.0295     0.00555556
      2              3      -0.552988              3      -0.533408         12.231      0.0240664
      3              6      -0.553859              3      -0.552988        10.9046      0.0211693
      4              9      -0.553859              3      -0.553859        10.9046      0.0210017
      5             12      -0.553859              3      -0.553859        6.82502      0.0165153
      6             15      -0.555301              3      -0.553859        7.93529      0.0179571
      7             18      -0.555301              3      -0.555301        7.93529      0.0176796
      8             21      -0.555631              3      -0.555301        5.84261      0.0180089
      9             24      -0.555631              3      -0.555631        5.62